In [1]:
# %load_ext tensorboard


# import tensorflow as tf
# import tensorflow_addons as tfa
# from tensorflow import keras
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from sklearn import preprocessing
# from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
# from sklearn import metrics
# from sklearn.metrics import classification_report


In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Convolution2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling1D
from keras import backend as K
from keras import backend
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time
import os
import psutil
import csv
from itertools import repeat
from PIL import Image
from numpy import asarray
from scipy.stats import entropy
import math

In [3]:
# try:
df = pd.read_csv("file0.csv",index_col=False, skipinitialspace=True)
# except:
#   df = pd.read_csv("C:\\Users\\jpath\\Project\\heart_cleaned.csv", index_col=False, skipinitialspace=True)

In [4]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [5]:
# df['HeartDisease'] = df['HeartDisease'].map({'No': 'Yes', 'Yes': 'No'})
# df.head()

In [6]:
# # Calculate the number of rows to flip (60% of the total rows)
# num_rows_to_flip = int(1.0 * len(df))

# # Randomly select rows to flip
# rows_to_flip = np.random.choice(df.index, num_rows_to_flip, replace=False)

# # Flip the labels for the selected rows
# df.loc[rows_to_flip, 'HeartDisease'] = df.loc[rows_to_flip, 'HeartDisease'].map({'No': 'Yes', 'Yes': 'No'})

# # Display the first few rows of the modified DataFrame
# df.head()

In [7]:
df = df.replace('?', np.nan)
df[pd.isnull(df).any(axis=1)].shape

(0, 18)

In [8]:
print(df.shape)

#'number_outpatient', ,'number_inpatient'

(29999, 18)


In [9]:
df['AgeCategory'].value_counts()  

65-69          3236
60-64          3082
70-74          2973
55-59          2600
80 or older    2305
50-54          2296
75-79          2159
45-49          2093
35-39          1930
40-44          1897
18-24          1894
30-34          1858
25-29          1676
Name: AgeCategory, dtype: int64

In [10]:
# categorizing age
#age <50,>50
df.loc[df['AgeCategory']== '18-24','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '25-29','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '30-34','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '35-39','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '40-44','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '45-49','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '50-54','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '55-59','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '60-64','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '65-69','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '70-74','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '75-79','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '80 or older','AgeCategory'] = 1;

df['AgeCategory'] = df['AgeCategory'].astype(np.int32)

In [11]:
df['AgeCategory'].value_counts()

1    18651
0    11348
Name: AgeCategory, dtype: int64

In [12]:
df['Race'].value_counts()

White                             21371
Hispanic                           3673
Black                              2137
Other                              1048
American Indian/Alaskan Native     1047
Asian                               723
Name: Race, dtype: int64

In [13]:
#converting race to binary

df.loc[df['Race']== 'White','Race'] = 'White';
df.loc[df['Race']== 'Black','Race'] = 'Non-White';
df.loc[df['Race']== 'Hispanic','Race'] = 'Non-White';
df.loc[df['Race']== 'Other','Race'] = 'Non-White';
df.loc[df['Race']== 'Asian','Race'] = 'Non-White';
df.loc[df['Race']== 'American Indian/Alaskan Native','Race'] = 'Non-White';

In [14]:
df['Race'].value_counts()

White        21371
Non-White     8628
Name: Race, dtype: int64

In [15]:
df['Sex'].value_counts()

Female    15784
Male      14215
Name: Sex, dtype: int64

In [16]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,1,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,1,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,1,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,1,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,0,White,No,Yes,Very good,8.0,No,No,No


In [17]:
#Normalization
normalize_columns = ['BMI','PhysicalHealth', 'MentalHealth', 'SleepTime']

In [18]:
def show_values(columns):
  for column in columns:
    max_val = df[column].max()
    min_val = df[column].min()
    mean_val = df[column].mean()
    var_val = df[column].var()
    print(column +': values=['+str(min_val)+','+str(max_val)+'] , mean='+str(mean_val)+' , var='+str(var_val))

In [19]:
show_values(normalize_columns)

BMI: values=[12.4,83.0] , mean=28.059314643821466 , var=39.640557674148226
PhysicalHealth: values=[0.0,30.0] , mean=3.6536551218373945 , var=67.38380814439509
MentalHealth: values=[0.0,30.0] , mean=3.9770325677522584 , var=62.965036850445784
SleepTime: values=[1.0,24.0] , mean=7.138804626820894 , var=2.1886131586740567


In [20]:
def normalize(columns):
  scaler = preprocessing.StandardScaler()
  df[columns] = scaler.fit_transform(df[columns])
normalize(normalize_columns)
show_values(normalize_columns)

BMI: values=[-2.4871965767791018,8.726326001610861] , mean=-8.413106428280529e-16 , var=1.0000333355556932
PhysicalHealth: values=[-0.4450994050453217,3.209592049424672] , mean=0.0 , var=1.0000333355557631
MentalHealth: values=[-0.5012064544060127,3.2795505235733122] , mean=1.231648463599627e-17 , var=1.0000333355560176
SleepTime: values=[-4.149602984755611,11.397539244269044] , mean=-1.3927101857626552e-16 , var=1.0000333355559528


In [21]:
categorical_columns = ['Smoking', 'AlcoholDrinking','Stroke', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'Asthma', 'KidneyDisease',  'SkinCancer']
label_column = ['HeartDisease']

# Smoking	AlcoholDrinking	Stroke	PhysicalHealth	MentalHealth	DiffWalking	Sex	AgeCategory	Race	Diabetic	PhysicalActivity	GenHealth	SleepTime	Asthma	KidneyDisease	SkinCancer

In [22]:
def show_unique_values(columns):
  for column in columns:
    uniq = df[column].unique().tolist()
    print(column+ " has "+ str(len(uniq)) +" values" + " : " + str(uniq))
show_unique_values(categorical_columns)
show_unique_values(label_column)

Smoking has 2 values : ['Yes', 'No']
AlcoholDrinking has 2 values : ['No', 'Yes']
Stroke has 2 values : ['No', 'Yes']
DiffWalking has 2 values : ['No', 'Yes']
Sex has 2 values : ['Female', 'Male']
AgeCategory has 2 values : [1, 0]
Race has 2 values : ['White', 'Non-White']
Diabetic has 4 values : ['Yes', 'No', 'No, borderline diabetes', 'Yes (during pregnancy)']
PhysicalActivity has 2 values : ['Yes', 'No']
GenHealth has 5 values : ['Very good', 'Fair', 'Good', 'Poor', 'Excellent']
Asthma has 2 values : ['Yes', 'No']
KidneyDisease has 2 values : ['No', 'Yes']
SkinCancer has 2 values : ['Yes', 'No']
HeartDisease has 2 values : ['No', 'Yes']


In [23]:
def convert_to_onehot(data,columns):
  dummies = pd.get_dummies(data[columns])
  data = data.drop(columns, axis=1)
  data = pd.concat([data, dummies], axis=1)
  return data

In [24]:
df = convert_to_onehot(df,categorical_columns)
df = convert_to_onehot(df,label_column)

In [25]:
df.head()

,BMI,PhysicalHealth,MentalHealth,SleepTime,AgeCategory,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,Stroke_No,...,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes,HeartDisease_No,HeartDisease_Yes
0,-1.820103,-0.079630,3.279551,-1.445752,1,0,1,1,0,1,...,0,1,0,1,1,0,0,1,1,0
1,-1.226072,-0.445099,-0.501206,-0.093827,1,1,0,1,0,0,...,0,1,1,0,1,0,1,0,1,0
2,-0.234962,1.991362,3.279551,0.582136,1,0,1,1,0,1,...,0,0,0,1,1,0,1,0,1,0
3,-0.611393,-0.445099,-0.501206,-0.769789,1,1,0,1,0,1,...,0,0,1,0,1,0,0,1,1,0
4,-0.690809,2.965946,-0.501206,0.582136,0,1,0,1,0,1,...,0,1,1,0,1,0,1,0,1,0


In [26]:
print(df.shape)

(29999, 36)


In [27]:
from sklearn.model_selection import train_test_split

s_names = ['HeartDisease_Yes', 'HeartDisease_No']
# s_names = ['HeartDisease']
x_data = df.drop(s_names, axis=1)
y_labels = df[s_names]

print(x_data.shape, y_labels.shape)
xClients, X_test, yClients, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=123)


df.shape

(29999, 34) (29999, 2)


(29999, 36)

In [28]:

algoName='CNN'
x_shape, y_shape =x_data.shape[1], y_labels.shape[1]
mean_shape = (x_shape + y_shape) // 2
#outputClasses=len(set(y_labels))
outputClasses=y_labels.shape[1]
def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    recall=recall_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    
    
    return accuracy, precision, recall, f1Score

verbose, epochs, batch_size = 0, 10, 64
activationFun='relu'
optimizerName='Adam'
def createDeepModel():
    model = Sequential()
    
    if(algoName=='CNN'):    
        model.add(Conv1D(filters=5, kernel_size=3, activation=activationFun,input_shape = (x_shape, 1))) # changed x_data.shape[1] to x_shape
        model.add(Dropout(0.1))
        model.add(MaxPooling1D(pool_size=3))
        
        model.add(Conv1D(filters=5, kernel_size=3, activation=activationFun))
        model.add(Dropout(0.05))
        model.add(MaxPooling1D(pool_size=3))
        
        model.add(Flatten())  # Uncommented this line
        model.add(Dense(x_shape, activation=activationFun))
        model.add(Dense(mean_shape, activation=activationFun))
        model.add(Dense(50, activation=activationFun))
        model.add(Dense(outputClasses, activation='sigmoid'))
        model.compile(loss='mean_squared_error', optimizer=optimizerName, metrics=['accuracy'])

    return model
def predictTestData(yPredict, y_test): ##
    #Converting predictions to label
    
    print("yPredict",len(yPredict))
    pred = list()
    for i in range(len(yPredict)):
        pred.append(np.argmax(yPredict[i])) ###i
    #Converting one hot encoded test label to label
    test = list()
    for i in range(len(y_test)):
        test.append(np.argmax(y_test[i]))
    return my_metrics(test, pred)

def sumOfWeights(weights):
    return sum(map(sum, weights))

def getWeights(model):
    allLayersWeights=deepModel.get_weights()
    return allLayersWeights


    
# Initially train central deep model
deepModel=createDeepModel()

In [29]:
def getWeights(deepModel):
    allLayersWeights = deepModel.get_weights()
    return allLayersWeights
for layer in deepModel.layers:
    weights = layer.get_weights() # list of numpy arrays
    if len(weights) > 0:
        print("Layer:", layer.name)
        print("Weights:", weights[0]) # weights array
        print("Biases:", weights[1]) # bias array

Layer: conv1d
Weights: [[[-0.33565846  0.3901654  -0.17364305 -0.09361604 -0.4022693 ]]

 [[ 0.06859231  0.507478    0.56673217  0.32284868  0.01990545]]

 [[-0.49848697 -0.31414333 -0.1603311  -0.22081429  0.55344105]]]
Biases: [0. 0. 0. 0. 0.]
Layer: conv1d_1
Weights: [[[-0.25542393 -0.19328997  0.1691879   0.4050662   0.2872175 ]
  [-0.23064122 -0.24490644  0.21559024 -0.16382751  0.3300885 ]
  [-0.01879781 -0.13078216  0.32414943  0.17961264  0.09550613]
  [ 0.13191557  0.2974558  -0.33801225  0.19568205  0.09179831]
  [-0.08447236  0.1927607   0.42049897  0.3462013  -0.08377609]]

 [[-0.25449246 -0.08621866  0.0598805  -0.28842077  0.02536073]
  [ 0.02227068  0.38931453 -0.12243691 -0.4127333   0.17503035]
  [-0.24214242 -0.37267923  0.31765908 -0.15370098 -0.11670044]
  [ 0.01144299  0.07091892 -0.1324859   0.10641813 -0.43042105]
  [-0.3435585   0.1748603  -0.30298585 -0.27188468 -0.2257541 ]]

 [[ 0.04283759  0.08144337 -0.03854054  0.08662391  0.16009021]
  [-0.13360587  0.158

In [30]:
getWeights(deepModel)

[array([[[-0.33565846,  0.3901654 , -0.17364305, -0.09361604,
          -0.4022693 ]],
 
        [[ 0.06859231,  0.507478  ,  0.56673217,  0.32284868,
           0.01990545]],
 
        [[-0.49848697, -0.31414333, -0.1603311 , -0.22081429,
           0.55344105]]], dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32),
 array([[[-0.25542393, -0.19328997,  0.1691879 ,  0.4050662 ,
           0.2872175 ],
         [-0.23064122, -0.24490644,  0.21559024, -0.16382751,
           0.3300885 ],
         [-0.01879781, -0.13078216,  0.32414943,  0.17961264,
           0.09550613],
         [ 0.13191557,  0.2974558 , -0.33801225,  0.19568205,
           0.09179831],
         [-0.08447236,  0.1927607 ,  0.42049897,  0.3462013 ,
          -0.08377609]],
 
        [[-0.25449246, -0.08621866,  0.0598805 , -0.28842077,
           0.02536073],
         [ 0.02227068,  0.38931453, -0.12243691, -0.4127333 ,
           0.17503035],
         [-0.24214242, -0.37267923,  0.31765908, -0.15370098,
      

In [31]:
deepModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 32, 5)             20        
                                                                 
 dropout (Dropout)           (None, 32, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 8, 5)              80        
                                                                 
 dropout_1 (Dropout)         (None, 8, 5)              0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 2, 5)             0         
 1D)                                                    

In [32]:
numOfIterations= 10
# modelLocation='C:\\Users\\PRAKHAR\\Documents\\GitHub\\dsci2023\\saved_model\\'+"final_model.h5"
accList, precList, recallList, f1List = [], [], [], []

deepModelAggWeights=[]
firstClientFlag=True

def updateServerModel(clientModel, clientModelWeight):
    global firstClientFlag
    for ind in range(len(clientModelWeight)):
        if(firstClientFlag==True):
            deepModelAggWeights.append(clientModelWeight[ind])            
        else:
            deepModelAggWeights[ind]=(deepModelAggWeights[ind]+clientModelWeight[ind])

def updateClientsModels():
    global clientsModelList
    global deepModel
    clientsModelList.clear()
    for clientID in range(numOfClients):
        m = keras.models.clone_model(deepModel)
        m.set_weights(deepModel.get_weights())
        clientsModelList.append(m)

In [33]:

# ------- 2. Prepare data for the client ----------

clientID = 0  # Only one client
xClientsList = [xClients]
yClientsList = [yClients]

index_i = yClientsList[clientID].index.tolist()
X_i = xClientsList[clientID].loc[index_i, :]
y_i = yClientsList[clientID]['HeartDisease_Yes'].tolist()
X_i['HeartDisease_Yes'] = y_i

y_i1 = yClientsList[clientID]['HeartDisease_No'].tolist()
X_i['HeartDisease_No'] = y_i1

model = load_model("final_model.h5")
clientsModelList = [model]

print("clientId:", clientID)
clientsModelList[clientID].fit(xClientsList[clientID], yClientsList[clientID], epochs=epochs, batch_size=batch_size)

# Save the client's model
clientsModelList[clientID].save("client_model.h5")


clientId: 0
Epoch 1/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0729 - accuracy: 0.9108
Epoch 2/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0730 - accuracy: 0.9108
Epoch 3/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0726 - accuracy: 0.9108
Epoch 4/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0727 - accuracy: 0.9107
Epoch 5/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0722 - accuracy: 0.9108
Epoch 6/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0727 - accuracy: 0.9111
Epoch 7/10
329/329 [==============================] - 1s 2ms/step - loss: 0.0729 - accuracy: 0.9105
Epoch 8/10
329/329 [==============================] - 1s 3ms/step - loss: 0.0729 - accuracy: 0.9108
Epoch 9/10
329/329 [==============================] - 1s 3ms/step - loss: 0.0726 - accuracy: 0.9114
Epoch 10/10
329/329 [==============================] - 1s 3ms/step - loss: 0.0727 - accu